In [1]:
import sys

print(sys.version)

3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]


# 1. 데코레이터

- 특정 함수나 클래스로 기존 함수나 클래스의 기능을 추가하는 경우 사용
- 보통 함수에 함수를 전달해서 새로운 기능을 처리 : 합성함수 처리


# 1.1 데코레이터 함수 정의

### 데코레이터 함수를 정의
- 하나의 매개변수는 함수를 받는다.
- 함수를 그대로 전달한다.

In [6]:
v = 100

In [7]:
v

100

In [1]:
def func(func_) :
    return func_

In [2]:
def add(x,y) :
    return x +y

In [3]:
add = func(add)

In [9]:
add(7, 8)

15

In [18]:
x = 10

In [19]:
x + 1

11

In [20]:
eval("x+1")

11

In [17]:
eval("add")

<function __main__.add(x, y)>

In [12]:
def arth(op,x,y):
    def add(x,y):
        return x+y
    def sub(x,y):
        return x-y
    return eval(op)(x,y)

In [13]:
arth("add",10,20), arth("sub",10,20)

(30, -10)

In [27]:
class Arth:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def arth(self,op):
        def add(x,y):
            return x+y
        def sub(x,y):
            return x-y
        return eval(op)(self.x,self.y)

In [28]:
a = Arth(10, 20)

In [29]:
a.arth("add")

30

### 데코레이터 함수 내에 내부 함수를 정의해서 새로운 기능 추가

- 함수를 전달 받아 지역변수로 처리
- 내부 함수를 반환한다.
- 내부 함수 내에서 저장된 함수를 실행한다. 

In [45]:
x = (x+y)+z

NameError: name 'y' is not defined

In [57]:
def dec(func):
    def inner(x,y):
        return x+y
    print("a")
    def new_func():
        xy = inner(10, 20)
        print('A')
        return func(xy)
    return new_func

In [58]:
def add(xy) :
    z = 10
    return xy + z

In [59]:
add = dec(add)

a


In [60]:
add()

A


40

In [61]:
add.__name__

'new_func'

In [62]:
add()

A


40

### 데코레이터 단순 표기법 
- 기존 함수에 @ 기호와 데코레이터 함수를 지정한다.
- 함수를 실행할 때는 실행함수 이름으로 실행
- 내부 함수가 실행함수 이름으로 반환되어 처리


In [63]:
@dec
def add_(xy) :
    z = 10
    return xy + z

a


In [64]:
add_()

A


40

## 1.2 데코레이터 실행함수 인자 전달 하기
- 실행함수 저장과 실행함수의 인자를 분리한다.
- 내부 함수는 실행함수의 인자를 받아서 저장된 실행함수를 실행한다.

In [69]:
def dec_1(func):
    print("a")
    def new_func(*args, **kwargs):
        print('A')
        return func(*args, **kwargs)
    return new_func

In [68]:
@dec_1
def add_1(x,y,z,a) :
    return x + y + z + a

a


In [70]:
add_1(100,200,300, 400)

A


1000

In [71]:
@dec_1
def sub(x,y):
    return x - y

a


In [72]:
sub(100, 90)

A


10

## 1.3  데코레이터 실행순서

- 데코레이터는 여러 개를 실행함수에 연결할 수 있다.
- 데코레이터 순서는 밑에 정의한 것부터 차례대로 구성된다.


In [14]:
def a(func):
    print("a")
    def new_func():
        print('A')
        func()
    return new_func


def b(func):
    print("b")
    def new_func():
        print('B')
        func()
    return new_func


def c(func):
    print("c")
    def new_func():
        print('C')
        func()
    return new_func

### 데코레이터 적용 순서 
- 로딩시 자동으로 데코레이터가 구성된다.

In [15]:
@a
@b 
@c # <- 우선 적용
def func():
    pass

c
b
a


### 실행순서 
- 실행슨서는 제일 위에 정의한 데코레이터부터 순차적으로 실행된다.


In [16]:
func()

A
B
C


In [17]:
def func_1():
    pass

func_1 = a(b(c(func_1))) # 데코레이터 문법 적용시키지 않고 동일한 함수를 생성

c
b
a


In [18]:
func_1()

A
B
C


# 2.  정보상실 문제 해결

- 데코레이터를 구성하면 내부함수가 전달되므로 저장된 실행함수의 정보가 외부에 전달되지 않는다.
- 외부함수의 정보를 전달하려면 함수 내의 메타정보를 갱신해야 한다.

* 덮어쓰기

`__name__`, `__module__`, `__qualname__`, `__doc__`, `__annotations__`

* 업데이트

`__dict__`

* 원본함수 저장

`__wrapped__`

## 2.1  내부 함수가 전달되어 기존 함수에 대한 정보가 없음

In [19]:
def outer(func):
    def inner(*args, **kwargs):
        print("함수가 실행됩니다.")
        return func(*args, **kwargs)
    return inner

In [20]:
@outer
def func_3(a, b, c):
    """3개의 숫자 또는 Sequence를 받아서 + 연산 후 리턴"""
    return a + b + c

In [21]:
func_3(1, 2, 3)

함수가 실행됩니다.


6

In [22]:
func_3('a', 'b', 'c')

함수가 실행됩니다.


'abc'

In [23]:
func_3.__name__

'inner'

In [24]:
func_3.__doc__

In [25]:
func_3.__module__

'__main__'

In [26]:
func_3.__qualname__

'outer.<locals>.inner'

In [27]:
func_3.__dict__

{}

## 2.2 wraps 구현
- 실행함수의 정부를 데코레이터를 사용해서 갱신하기


In [28]:
from functools import wraps

def dec(func):
    @wraps(func) # 메타 정보 관리
    def wrapper(*args, **kwargs): # *: 가변인자 / **: 키워드인자
        result = func(*args, **kwargs)
        return result
    return wrapper



In [29]:
@dec # @dec 사용하지 않을 경우: add = dec(add) ; def add(x, y): return x + y ;
def add(x, y):
    return x + y

In [30]:
@dec
def multiply(x, y):
    return x * y

In [31]:
print(add(5, 5))

10


In [32]:
add.__name__

'add'

In [33]:
print(multiply(4, 3))

12


In [34]:
multiply.__name__

'multiply'

## 2-3  더 깊은 데코레이터

- 데코레이터를 처리하는 경우 데코레이터에도 별도의 매개변수를 받을 수 있다.
- 이때 제일 밖의 함수는 데코레이터의 매개변수를 받고 그 내부 함수부터 함수를 받도록 처리한다.

In [35]:
def make_deco(value):
    def outer(func):
        def inner(*args, **kwargs):
            print(value)
            return func(*args, **kwargs)
        return inner
    return outer

In [36]:
@make_deco('!!!@#!@#!@#!@#$@#$%@!#$%@#$')
def func():
    print("func!!!")

In [37]:
func()

!!!@#!@#!@#!@#$@#$%@!#$%@#$
func!!!


# 3. 데코레이터 활용

- 특정 기능을 추가할 때 내부 함수에 기능을 추가해서 처리하도록 만든다.

### 타입 강제하는 데코레이터

```
@typecheck(int, int, int)
def sum(a, b, c):
    return a + b + c


sum(1, 2, 3) # 6 
sum('a', 'b', 'c') # TypeError
```

### 최상위 테코레이터 함수에 인자를 받아서 처리합니다.

In [38]:
def typecheck(*t_args):# t_args
    def outer(func):
        def inner(*args, **kwargs):
            for i, t_arg in enumerate(t_args):
                if t_arg != type(args[i]):
                    raise TypeError("정확하지 않은 타입입니다.")
            return func(*args, **kwargs)
        return inner
    return outer

In [39]:
@typecheck(int, int, int)
def sum(a, b, c):
    return a + b + c

In [40]:
sum(1, 2, 3)

6

### 최상위 테코레이터 함수에 인자를 받아서 처리합니다.

In [41]:
## 키워드인자들도 반응하도록 

def typecheck_1(*t_args, **t_kwargs):# t_args
    def outer(func):
        def inner(*args, **kwargs):
            # 위치인자 검증
            for i, t_arg in enumerate(t_args):
                if t_arg != type(args[i]):
                    raise TypeError("정확하지 않은 타입입니다.")
                    
            # 키워드인자 검증
            for key in t_kwargs:
                if t_kwargs[key] != kwargs[key].__class__:
                    raise TypeError("정확하지 않은 타입입니다.")
            
            return func(*args, **kwargs)
        return inner
    return outer

In [42]:
@typecheck_1(a=int, b=int, c=int)
def sum(a, b, c):
    return a + b + c

In [43]:
sum(a=3, b=2, c=3)

8